In [3]:
!pip install beautifulsoup4 

     |████████████████████████████████| 122kB 4.9MB/s eta 0:00:01


In [4]:
!pip install lxml

     |████████████████████████████████| 5.5MB 22.2MB/s eta 0:00:01


Importing all required libraries

In [5]:
from bs4 import BeautifulSoup
import requests
import lxml

Creating a class to BeautifulSoup class.
And then finding all the tables with class 'wikitable sortable'

In [6]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'html.parser')
tables = soup.find_all('table', class_='wikitable sortable')


Create a empty list and then append all the values with tag 'th'. This list contains the column names of dataframe

In [7]:
# Headers
headers = []
for i in tables:
    ths = i.find_all('th')
    for th in ths:
        headings = th.text.strip()
        headers.append(headings)

print(headers)

['Postal Code', 'Borough', 'Neighborhood']


Similarly, we create a list of lists that contains values of rows

In [8]:
values_raw = []  # Contains raw html text values from the page
for i in tables:
    ths = i.find_all('tr')
    for th in ths:
        values = th.text.strip()
        values_raw.append(values)


df_list = []  # Contains filtered values. Loop splits the list on charecter '\n' 
for i in range(1, len(values_raw)): 
    new = values_raw[i].split('\n')
    df_list.append(new)
    
    
tbr = []  # This list contains values that needs to be removed from original list. Since the NA lists has length of 3, we can loop to match the length ==3 
for k in df_list:
    if len(k) == 3:
        tbr.append(k)

for z in tbr:  # Loop to remove all the tbr values from original list
    df_list.remove(z)

# Our df_list now contains necessary values. But due tosplit it contains unwanted '' spaces. To remove that we can use pop.
for a in df_list:
    a.pop(1)
    a.pop(2)

df_list  # Final list of dataframe

[['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Regent Park, Harbourfront'],
 ['M6A', 'North York', 'Lawrence Manor, Lawrence Heights'],
 ['M7A', 'Downtown Toronto', "Queen's Park, Ontario Provincial Government"],
 ['M9A', 'Etobicoke', 'Islington Avenue, Humber Valley Village'],
 ['M1B', 'Scarborough', 'Malvern, Rouge'],
 ['M3B', 'North York', 'Don Mills'],
 ['M4B', 'East York', 'Parkview Hill, Woodbine Gardens'],
 ['M5B', 'Downtown Toronto', 'Garden District, Ryerson'],
 ['M6B', 'North York', 'Glencairn'],
 ['M9B',
  'Etobicoke',
  'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale'],
 ['M1C', 'Scarborough', 'Rouge Hill, Port Union, Highland Creek'],
 ['M3C', 'North York', 'Don Mills'],
 ['M4C', 'East York', 'Woodbine Heights'],
 ['M5C', 'Downtown Toronto', 'St. James Town'],
 ['M6C', 'York', 'Humewood-Cedarvale'],
 ['M9C',
  'Etobicoke',
  'Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood

In [9]:
import pandas as pd  

In [10]:
canada_df = pd.DataFrame(df_list, columns=headers)  # Turning the list to dataframe
canada_df.head()


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [11]:
canada_df.shape  # The shape of dataframe

(103, 3)

In [12]:
latlong_df = pd.read_csv("Geospatial_Coordinates.csv")

In [13]:
latlong_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
canada_df = canada_df.sort_values(by=['Postal Code'])
canada_df.drop(['level_0', 'index'], axis=1)

KeyError: "['level_0' 'index'] not found in axis"

In [ ]:
final_df = pd.concat([canada_df, latlong_df], axis=1)
final_df = final_df.drop(['level_0', 'index'], axis=1)
final_df

In [ ]:
final_df.head()

In [ ]:
final_df.to_csv('canada.csv')